In [17]:
import numpy as np
import xarray as xr
import torch
from pathlib import Path
import re
import time

In [18]:
input_file = '/Users/kramea/Documents/AIBEDO_dir/data_aibedo/compress.isosph.CESM2.historical.r1i1p1f1.Input.Exp8_fixed.nc'

In [19]:
output_file = '/Users/kramea/Documents/AIBEDO_dir/data_aibedo/compress.isosph.CESM2.historical.r1i1p1f1.Output.nc'

In [20]:
inDS = xr.open_dataset(input_file)
outDS = xr.open_dataset(output_file)
n_pixels = len(inDS.ncells)

In [21]:
in_vars = [ 'crelSurf_pre', 'crel_pre', 'cresSurf_pre', 'cres_pre', 'netTOAcs_pre', 'lsMask', 'netSurfcs_pre']
out_vars = ['tas_pre', 'psl_pre', 'pr_pre']

In [22]:
unet = torch.load('/Users/kramea/Documents/AIBEDO_dir/data_aibedo/unet_model_4.pt', 
                        map_location=torch.device('cpu'))

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
unet = unet.module.to(device)

In [25]:
unet.eval()

SphericalUNet(
  (encoder): Encoder(
    (pooling): IcosahedronPool()
    (enc_l5): SphericalChebBN2(
      (spherical_cheb_bn_1): SphericalChebBN(
        (spherical_cheb): SphericalChebConv(
          (chebconv): ChebConv()
        )
        (batchnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (spherical_cheb_bn_2): SphericalChebBN(
        (spherical_cheb): SphericalChebConv(
          (chebconv): ChebConv()
        )
        (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (enc_l4): SphericalChebBNPool(
      (pooling): IcosahedronPool()
      (spherical_cheb_bn): SphericalChebBN(
        (spherical_cheb): SphericalChebConv(
          (chebconv): ChebConv()
        )
        (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (enc_l3): SphericalChebBNPool(
      (pooling): IcosahedronPool()
      (spherical_c

In [26]:
modelfilename = Path(output_file).stem
p = re.compile('compress.isosph.(.*).historical.r1i1p1f1.Output')
modelname = p.findall(modelfilename)[0]

In [27]:
data_all = []
for var in in_vars:
    temp_data = np.reshape(np.concatenate(inDS[var].data, axis=0), [-1, n_pixels, 1])
    data_all.append(temp_data)
dataset_in = np.concatenate(data_all, axis=2)

In [28]:
data_all = []
for var in out_vars:
    temp_data = np.reshape(np.concatenate(outDS[var].data, axis=0), [-1, n_pixels, 1])
    data_all.append(temp_data)
dataset_out = np.concatenate(data_all, axis=2)

In [29]:
# timesteps to predict
inPredict = dataset_in[1000:1020]
outPredict = dataset_out[1000:1020]

In [30]:
before = time.perf_counter()
preds = unet(torch.Tensor(inPredict)) # Change this to the desired number of timesteps
after = time.perf_counter()
print("time taken to perform prediction", after - before)

time taken to perform prediction 11.01708124999999


In [31]:
#Predictions
pred_numpy = preds.detach().cpu().numpy()

In [32]:
np.save(("/Users/kramea/Documents/AIBEDO_dir/data_aibedo/"+ modelname + "_predictions.npy"), pred_numpy)
np.save(("/Users/kramea/Documents/AIBEDO_dir/data_aibedo/" + modelname + "_groundtruth.npy"), outPredict)